<div id="toc"> </div>

# Import lots of stuff

In [2]:
import itertools
import math
import re
import csv
import re;
import json
import os
import pandas as pd
import pylab as pyl
import nltk as nltk
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from pycorenlp import StanfordCoreNLP
%matplotlib inline
#enable longer display
pd.set_option('display.max_rows', 500)

# Preprocess data for analyses

## Import annotated file

In [65]:
d_raw = pd.read_csv('handTagged.csv')
d_nicki = (pd.read_csv('../../data/tangrams_unconstrained/old/oldTangrams.csv')
    .query('tangram != "*"')
    .drop('sender', 1)
    .rename(columns = {'tangram' : 'tangramRef'}))

# Drop time column
d = (d_raw
    .copy()
    .drop('time', 1)
    .query('tangramRef != "0"'))

# Result 1: Generate file for POS analysis

## Start the Stanford CoreNLP server

Before running this notebook, [get CoreNLP](http://nlp.stanford.edu/software/stanford-corenlp-full-2015-12-09.zip), go into its directory, and run

`java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer 9000`

If you're using port 9000 for something else, change that value and then change `PORT` in the next cell.

In [66]:
PORT = 9000

nlp = StanfordCoreNLP('http://localhost:{}'.format(PORT))

## Get part of speech

Todo: follow Will's advice to parse unicode...

In [69]:
def stanford_pos(text):
    """
    Parameters
    ----------
    text : str
       CoreNLP handles all tokenizing, at the sentence and word level.
       
    Returns
    -------
    list of tuples (str, str)
       The first member of each pair is the word, the second its POS tag.          
    """
    try:
        ann = nlp.annotate(
            text, 
            properties={'annotators': 'pos', 
                        'outputFormat': 'json'})
        lemmas = []
        for sentence in ann['sentences']:
            for token in sentence['tokens']:
                lemmas.append((token['word'], token['pos']))
    except Exception as e:
        print(text + ": cannot parse")
        lemmas = []
    return lemmas

In [70]:
def stanford_constituency(text):
    """
    Parameters
    ----------
    text : str
       CoreNLP handles all tokenizing, at the sentence and word level.
       
    Returns
    -------
    list of tuples (str, str)
       The first member of each pair is the word, the second its POS tag.          
    """
    try:
        ann = nlp.annotate(
            text, 
            properties={'annotators': 'parse', 
                        'outputFormat': 'json'})
        lemmas = []
        for sentence in ann['sentences']:
            for element in sentence['enhancedPlusPlusDependencies']:
                lemmas.append(element)
    except Exception as e:
        print(text,": cannot parse")
        lemmas = []
    return lemmas

In [106]:
from nltk.stem.wordnet import WordNetLemmatizer

LEMMATIZER = WordNetLemmatizer()

def is_comp_sup(word, pos, tags, check_lemmatizer=False):
    """
    Parameters
    ----------
    word, pos : str, str
        The lemma.
    
    tags : iterable of str
        The tags considered positive evidence for comp/sup morphology.
       
       
    check_lemmatizer : bool
        If True, then if the `pos` is in `tags`, we also check that
        `word` is different from the lemmatized version of word
        according to WordNet, treating it as an adjective. This 
        could be used to achieve greater precision, perhaps at the
        expense of recall.
       
    Returns
    -------
    bool       
    """
    if pos not in tags:
        return False
    if check_lemmatizer and LEMMATIZER.lemmatize(word, 'a') == word:
        return False
    return True

def is_noun(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'NN', 'NNS', 'NNP', 'NNPS'}, check_lemmatizer=check_lemmatizer)

def is_prep(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'IN'}, check_lemmatizer=check_lemmatizer)

def is_verb(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'MD', 'VB', 'VBZ', 'VBP', 'VBD', 'VBN', 'VBG'}, check_lemmatizer=check_lemmatizer)

def is_det(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'DT', 'WDT'}, check_lemmatizer=check_lemmatizer)

def is_pronoun(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'PRP', 'PRP$', 'WP', 'WP$'}, check_lemmatizer=check_lemmatizer)

def is_adjective(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'JJ', 'JJR', 'JJS'}, check_lemmatizer=check_lemmatizer)

def is_adverb(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'RB', 'RBR', 'RBS', 'RP', 'WRB'}, check_lemmatizer=check_lemmatizer)

def is_num(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'CD'}, check_lemmatizer=check_lemmatizer)

def is_other(word, pos, check_lemmatizer=False):
    return is_comp_sup(
        word, pos, {'EX', 'FW', 'LS', 'PDT', 'POS', 'SYM', 'TO', 'UH'}, check_lemmatizer=check_lemmatizer)

def is_compound(tags) :
    dependentClauseTags = {'csubj', 'ccomp', 'xcomp', 'advcl'}
    return sum([tag in dependentClauseTags for tag in tags]) > 0

def is_adjectival(tags)  :
    return sum([tag in {'acl', 'acl:relcl'} for tag in tags]) > 0

In [107]:
# A lemma is a (word, pos) tag pair.
d['lemmas'] = [stanford_pos(text) for text in d['contents']]

picture 12 is like a guy with both hands in the air, kind of like  ¯_(ツ)_/¯ : cannot parse
11 is  ¯_(ツ)_/¯ : cannot parse
12 is  ¯_(ツ)_/¯ : cannot parse
7 is  ¯_(ツ)_/¯ : cannot parse
9 is  ¯_(ツ)_/¯ : cannot parse
The crouching guy´s feet are a triangle?: cannot parse
2 is  ¯_(ツ)_/¯ : cannot parse


In [108]:
d['tokens'] = [[element[0] for element in l] for l in d['lemmas']]
d['pos'] = [[element[1] for element in l] for l in d['lemmas']]

In [109]:
d['numWords'] = [pd.value_counts(words).sum() for words in d['tokens']]

In [110]:
d['tags'] = [[w['dep'] for w in stanford_constituency(text)] for text in d['contents']]

Here's what I have in order: bunny ears (1), arms in air (2), standing on one leg, other leg to the left (3), robe pointing left (4), tilted square, rectangle stuff cut out of it (5), standing on one leg, other leg to the right (6), sitting with knees (7), sitting without knees (8), spike coming out of stomach, straight back (9), tilted square, rectangle with no stuff cut out (10), spike coming out of both back and stomach (11) and funky chicken (12). : cannot parse
picture 12 is like a guy with both hands in the air, kind of like  ¯_(ツ)_/¯ : cannot parse
11 is  ¯_(ツ)_/¯ : cannot parse
12 is  ¯_(ツ)_/¯ : cannot parse
7 is  ¯_(ツ)_/¯ : cannot parse
9 is  ¯_(ツ)_/¯ : cannot parse
The crouching guy´s feet are a triangle?: cannot parse
2 is  ¯_(ツ)_/¯ : cannot parse


In [111]:
d['clauses'] = [is_compound(tags) for tags in d['tags']]
d['acl'] = [is_adjectival(tags) for tags in d['tags']]

## Get counts for each POS label

In [112]:
d['nouns'] = [sum([1 if is_noun(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['prepositions'] = [sum([1 if is_prep(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['verbs'] = [sum([1 if is_verb(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['determiners'] = [sum([1 if is_det(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['pronouns'] = [sum([1 if is_pronoun(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['adjectives'] = [sum([1 if is_adjective(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['adverbs'] = [sum([1 if is_adverb(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['numbers'] = [sum([1 if is_num(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]
d['others'] = [sum([1 if is_other(*lem) else 0 for lem in lemmas])
                     for lemmas in d['lemmas']]

In [117]:
d

,gameid,roundNum,sender,contents,tangramRef,lemmas,tokens,pos,numWords,clauses,...,prepositions,verbs,determiners,pronouns,adjectives,adverbs,numbers,others,tags,acl
2,0955-2,1,director,number 1 is a guy that looks like he is prayin...,L,"[(number, NN), (1, CD), (is, VBZ), (a, DT), (g...","[number, 1, is, a, guy, that, looks, like, he,...","[NN, CD, VBZ, DT, NN, WDT, VBZ, IN, PRP, VBZ, ...",17,False,...,2,4,4,1,0,0,1,1,"[ROOT, nsubj, nummod, cop, det, nsubj, ref, ac...",True
4,0955-2,1,director,number 2 is a man who looks like he's standing...,A,"[(number, NN), (2, CD), (is, VBZ), (a, DT), (m...","[number, 2, is, a, man, who, looks, like, he, ...","[NN, CD, VBZ, DT, NN, WP, VBZ, IN, PRP, VBZ, V...",26,False,...,4,4,4,3,0,0,2,1,"[ROOT, nsubj, nummod, cop, det, nsubj, ref, ac...",True
6,0955-2,1,director,number 3 is a man with his hands in the air lo...,C,"[(number, NN), (3, CD), (is, VBZ), (a, DT), (m...","[number, 3, is, a, man, with, his, hands, in, ...","[NN, CD, VBZ, DT, NN, IN, PRP$, NNS, IN, DT, N...",16,False,...,3,4,2,2,0,0,1,0,"[ROOT, nsubj, nummod, cop, det, case, nmod:pos...",True
7,0955-2,1,director,number 4 i sa man with his foot parallel to th...,I,"[(number, NN), (4, CD), (i, FW), (sa, FW), (ma...","[number, 4, i, sa, man, with, his, foot, paral...","[NN, CD, FW, FW, NN, IN, PRP$, NN, NN, TO, DT,...",21,False,...,2,0,3,2,0,0,1,4,"[ROOT, nummod, amod, compound, dep, case, nmod...",False
8,0955-2,1,director,number 5 is a person on ground that doesn't lo...,G,"[(number, NN), (5, CD), (is, VBZ), (a, DT), (p...","[number, 5, is, a, person, on, ground, that, d...","[NN, CD, VBZ, DT, NN, IN, NN, WDT, VBZ, RB, VB...",14,False,...,2,3,3,0,0,1,1,0,"[ROOT, nsubj, nummod, cop, det, nsubj, case, n...",True
9,0955-2,1,director,number 6 looks similar but with big pointy boobs,B,"[(number, NN), (6, CD), (looks, VBZ), (similar...","[number, 6, looks, similar, but, with, big, po...","[NN, CD, VBZ, JJ, CC, IN, JJ, JJ, NNS]",9,True,...,1,1,0,0,3,0,1,0,"[ROOT, nsubj, nummod, xcomp, cc, case, amod, c...",False
10,0955-2,1,director,number 7 looks like a person praying,D,"[(number, NN), (7, CD), (looks, VBZ), (like, I...","[number, 7, looks, like, a, person, praying]","[NN, CD, VBZ, IN, DT, NN, VBG]",7,False,...,1,2,1,0,0,0,1,0,"[ROOT, nsubj, nummod, case, det, nmod:like, acl]",True
11,0955-2,1,director,number 8 looks like a blob with no arms,H,"[(number, NN), (8, CD), (looks, VBZ), (like, I...","[number, 8, looks, like, a, blob, with, no, arms]","[NN, CD, VBZ, IN, DT, NN, IN, DT, NNS]",9,False,...,2,1,2,0,0,0,1,0,"[ROOT, nummod, dep, case, det, nmod:like, case...",False
12,0955-2,1,director,number 9 looks like a rabbit sitting on the groud,E,"[(number, NN), (9, CD), (looks, VBZ), (like, I...","[number, 9, looks, like, a, rabbit, sitting, o...","[NN, CD, VBZ, IN, DT, NN, VBG, IN, DT, NN]",10,False,...,2,2,2,0,0,0,1,0,"[ROOT, nsubj, nummod, case, det, nmod:like, ac...",True
13,0955-2,1,director,number 10 looks like a woman praying with her ...,K,"[(number, NN), (10, CD), (looks, VBZ), (like, ...","[number, 10, looks, like, a, woman, praying, w...","[NN, CD, VBZ, IN, DT, NN, VBG, IN, PRP$, NNS, ...",13,False,...,2,2,2,1,0,0,1,1,"[ROOT, nsubj, nummod, case, det, nmod:like, ac...",True


## Export to csv for plotting in R

In [116]:
(d.drop(["lemmas", "contents", "tokens"], 1)
 .to_csv("posTagged.csv", index = False))

# Result 2: Calculate indicator words for tangrams/rounds

## First, get list of words in first round

In [17]:
# Filter down to first round
d_round1 = d[d['roundNum'] == 1]

# Pull out all tokens and collapse into count dict
tokenDict = Counter([item for sublist in d_round1['tokens'].tolist()
                     for item in sublist])

# Pull out all words that occur more than once
wordList = [word for (word,count) in tokenDict.items() if count > 1 and not word.isdigit()]
print(wordList[0:10])
print(len(wordList))

# Get POS map; will be longer because it doesn't require count > 1, but it doesn't matter
POSdict = {word: POS for lemma in d_round1['lemmas'] for (word, POS) in lemma}
print(len(POSdict.keys()))

[u'gown', u'all', u'pointing', u'hanging', u'go', u'chair', u'row', u'cliff', u'to', u'tail']
449
940


## Get list of all game ids

In [18]:
gameidList = pd.unique(d.gameid.ravel()).tolist()
print(gameidList[0:5])

['0955-2', '4459-4', '4895-5', '5273-c', '9475-3']


## Get list of all tangram names

In [19]:
tangramList = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
print(tangramList)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']


## Helper functions to select words & counts

In [20]:
def getWordCounts(df, gameid, roundNum, tangram = None) :
    roundCond = 'roundNum == ' + roundNum
    gameidCond = 'gameid == "' + gameid + '"'
    if(tangram is not None) :
        tangramCond = 'tangramRef == "' + tangram + '"'
        cond = " and ".join((roundCond, gameidCond, tangramCond))
    else :
        cond = " and ".join((roundCond, gameidCond))
    relevantRow = df.query(cond)
    return Counter([item for sublist in relevantRow['tokens'].tolist() 
                    for item in sublist])

#creates mini dataframe that grabs the words used in round n for a given tangram and gameid
def selectTangramRoundWords(df, tangram, roundNum, gameid):
    wordCounts = getWordCounts(df, gameid, roundNum, tangram)
    return list(wordCounts.keys())

## Helper function to compute PMIs

Note that merging is really costly -- if we need to speed it up, this might be the first target. Can also vectorize the log operation.

In [21]:
#returns a table with the all words above 0 PMI and their counts for a given tangram
#calculate the probability for words given tangram A ------ p(x|y)
def makeMyPMI(df, tangram, roundNum, gameid, totals):

    # count words w/in tangram
    tangramCounts = getWordCounts(df, gameid, roundNum, tangram)

    #total number of words 
    tangramNumWords = (1 if sum(tangramCounts.values()) == 0 
                       else sum(tangramCounts.values()))

    #dataframe to compare 
    indicatorDF = pd.merge(pd.DataFrame(list(tangramCounts.items()), columns=['word', 'count']),
                           pd.DataFrame(list(totals["counts"].items()), columns=['word', 'totalCount']),
                           on='word', how = 'inner')

    #calculate PMI without log first. Having trouble with float issues. 
    indicatorDF['roughPMI'] = ((indicatorDF['count']/tangramNumWords)
                                / (indicatorDF['totalCount']/totals["numWords"]))
    indicatorDF['logPMI'] = [math.log10(num) for num in indicatorDF['roughPMI']]
    
    #remove column rough PMI
    indicatorDF = indicatorDF.drop('roughPMI', 1)
    
    return indicatorDF

## Write out PMIs & matching rates for all words

Note that we do a sloppy optimization by only computing total counts once and only when necessary 

In [59]:
def memoize(d, gameid, counts) : 
    if "counts" not in counts : 
        counts["counts"] = getWordCounts(d, gameid, "1")
        counts["numWords"] = float(sum(counts["counts"].values()))
        return counts
    else :
        return counts

with open('matchAndPMI.csv', 'a', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['word', 'POS', 'match', 'pmi', 'total'])
    for word in wordList :
        print(word + ":" + POSdict[word])
        pmi = 0
        match = 0
        total = 0
        for gameid in gameidList:  
            memoizedCounts = {}
            for tangram in tangramList:
                memoizedCounts = memoize(d, gameid, memoizedCounts)
                round1WordList = selectTangramRoundWords(d, tangram, "1", gameid)
                total = total + 1 if word in round1WordList else total
                if word in round1WordList :
                    PMI_df = makeMyPMI(d, tangram, "1", gameid, memoizedCounts)
                    pmi = pmi + PMI_df[PMI_df['word'] == word]['logPMI'].tolist()[0]
                    round6WordList = selectTangramRoundWords(d, tangram, "6", gameid)
                    match = (match + 1 if (word in round1WordList and word in round6WordList)
                             else match)
        writer.writerow([word, POSdict[word], float(match) / float(total), pmi/total, total])


wearing:VBG
bottom:NN
them:PRP
rectangular:JJ
beanbag:NN
dancer:NN
angle:NN
limbs:NNS
how:WRB
not:RB
falling:VBG
by:IN
shaped:VBN
possibly:RB
on:IN
see:VB
other:JJ
blob:NN
shape:NN
kneel:NN
all:DT
right:NN
do:VB
ill:RB
'll:MD
kick:VBP
pacman:NN
have:VBP
rolling:VBG
correct:JJ
first:RB
sides:NNS
rock:NN
are:VBP
doing:VBG
again:RB
really:RB
describe:VB
meant:VBN
i:FW
small:JJ
stuck:VBD
dog:NN
face:NN
so:RB
can:MD
we:PRP
position:NN
tail:NN
number:NN
pretty:RB
pope:NN
vertical:JJ
towards:IN
his:PRP$
lower:JJR
me:PRP
bowl:NN
arm:NN
nose:NN
same:JJ
they:PRP
start:VB
long:JJ
raised:VBD
robe:NN
side:NN
thats:NNS
too:RB
No.:NN
cube:NN
offering:NN
rabbit:NN
completely:RB
stretch:VBP
tray:NN
no:DT
without:IN
begging:VBG
half:NN
person:NN
second:JJ
They:PRP
!:.
yes:RB
candle:NN
PASS:NN
tip:NN
there:RB
duck:NN
reminds:VBZ
flag:NN
portruding:VBG
for:IN
an:DT
surface:NN
somewhat:RB
bunny:NN
weird:JJ
seems:VBZ
#:#
actually:RB
spread:VBN
leaned:VBD
?:.
For:IN
'':''
both:DT
does:VBZ
into:IN
those:DT
ha

Also write out raw word counts on each round (so we can see what most likely words to be dropped are).

In [23]:
for row in d :
    print row

gameid
roundNum
sender
contents
tangramRef
lemmas
tokens
pos
numWords
nouns
prepositions
verbs
determiners
pronouns
adjectives
adverbs
numbers
others


## Compute unigrams and bigrams

In [44]:
from nltk import bigrams

d['bigrams'] = [list(bigrams(l)) for l in d['tokens']]
bigramDict = Counter([item for sublist in d['bigrams'].tolist()
                     for item in sublist])
bigramList = [bigram for (bigram,count) in bigramDict.items() if count > 5]

In [45]:
def getBigramCounts(df, gameid, roundNum, tangram = None) :
    roundCond = 'roundNum == ' + roundNum
    gameidCond = 'gameid == "' + gameid + '"'
    if(tangram is not None) :
        tangramCond = 'tangramRef == "' + tangram + '"'
        cond = " and ".join((roundCond, gameidCond, tangramCond))
    else :
        cond = " and ".join((roundCond, gameidCond))
    relevantRow = df.query(cond)
    return Counter([item for sublist in relevantRow['bigrams'].tolist() 
                    for item in sublist])

with open('bigramCounts.csv', 'a') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['gameid', 'roundNum', 'word', 'count'])
    for gameid in gameidList:  
        for roundNum in ['1', '2', '3', '4', '5', '6'] :
            counts = getBigramCounts(d, gameid, roundNum)
            for bigram in bigramList :
                writer.writerow([gameid, roundNum, ' '.join(bigram), counts[bigram]])

In [ ]:
## bootstrap analysis (might want to move to R)

TODO: exclude numbers earlier in the pipeline, 

TODO: don't average over matches and pmis

In [77]:
#grab words with highestPMI for a given tangram/gameid
def highestPMIWords(d, tangram, roundNum, gameid):
    allTangramCounts = {}
    allTangramCounts['counts'] = getWordCounts(d, gameid, "1")
    allTangramCounts['numWords'] = float(sum(allTangramCounts["counts"].values()))

    PMIdf = (makeMyPMI(d, tangram, roundNum, gameid, allTangramCounts))

    # Remove numbers
    PMIdf['POS'] = [POSdict[word] for word in PMIdf['word']]
    PMIdf = PMIdf.query('POS != "CD"'.format())

    #if PMIdf has words, pull out max values, it is empty return it as is
    if len(PMIdf.index) > 0:
        PMI_values = PMIdf.logPMI.unique()
        maxPMI = PMI_values.max()
        PMIdf = PMIdf.loc[PMIdf['logPMI'] == maxPMI]
        PMIdfword = PMIdf['word']
        return PMIdfword.tolist()
    else: 
        return PMIdf

with open('PMIbootstrap.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['sampleNum', 'tangram', 'gameid', 'numCandidates', 'match', 'highest'])
    for gameid in gameidList :
        for tangram in tangramList :
            round1Words = selectTangramRoundWords(d, tangram, "1", gameid)
            if len(round1Words) > 0:
                # First, write highest PMI match
                highPMIWords = highestPMIWords(d, tangram, "1", gameid)
                round6Words = selectTangramRoundWords(d, tangram, "6", gameid)
                match = np.mean([1 if word in round6Words else 0 for word in highPMIWords ])
                writer.writerow([0, tangram, gameid, len(highPMIWords), match, "highest"])

                # Next, take a bunch of null samples
                for i in range(numSamples) :
                    randomWord = np.random.choice(round1Words)
                    match = np.mean([1 if randomWord in round6Words else 0])
                    writer.writerow([i + 1, tangram, gameid, 1, match, "null"])